In [1]:
# --- KDD: Selection, Preprocessing, Transformation, Data Mining, Interpretation ---
import os, urllib.request, zipfile, io, warnings
warnings.filterwarnings("ignore")

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE

os.makedirs("artifacts", exist_ok=True); os.makedirs("data", exist_ok=True); os.makedirs("artifacts/figs", exist_ok=True)

# SELECTION: Kaggle credit card fraud (European cardholders, PCA'ed features)
URL = "https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv"
path = "data/creditcard.csv"
if not os.path.exists(path):
    urllib.request.urlretrieve(URL, path)
df = pd.read_csv(path)

# PREPROCESSING
y = df["Class"].values
X = df.drop(columns=["Class"])

# TRANSFORMATION
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# DATA MINING (two angles):
# 1) Supervised (if labels available): handle imbalance with SMOTE + Logistic Regression
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

sm = SMOTE(random_state=42)
X_tr, y_tr = sm.fit_resample(X_train, y_train)

clf = LogisticRegression(max_iter=500, class_weight=None)
clf.fit(X_tr, y_tr)
proba = clf.predict_proba(X_test)[:,1]

print("Supervised AUC:", roc_auc_score(y_test, proba))
print("Average Precision:", average_precision_score(y_test, proba))

# 2) Unsupervised anomaly detection (when labels scarce)
iso = IsolationForest(n_estimators=400, contamination=0.0015, random_state=42)
iso.fit(X_train)
anom_score = -iso.decision_function(X_test)
print("IForest AUC (inference only):", roc_auc_score(y_test, anom_score))

# INTERPRETATION
from sklearn.inspection import permutation_importance
perm = permutation_importance(clf, X_test, y_test, n_repeats=5, random_state=42, scoring="average_precision")
imp = pd.Series(perm.importances_mean).sort_values(ascending=False)[:15]
plt.figure(figsize=(6,5)); imp[::-1].plot(kind="barh"); plt.title("Permutation importance (top15)"); plt.tight_layout()
plt.savefig("artifacts/figs/importance_kdd.png"); plt.close()

# Save artifacts
import joblib, json
joblib.dump({"scaler": scaler, "clf": clf}, "artifacts/fraud_model.pkl")
with open("artifacts/metrics.json","w") as f: json.dump({
    "auc_supervised": float(roc_auc_score(y_test, proba)),
    "ap_supervised": float(average_precision_score(y_test, proba))
}, f, indent=2)
print("Saved artifacts/fraud_model.pkl")


Supervised AUC: 0.970987885165321
Average Precision: 0.7244601740257454
IForest AUC (inference only): 0.9540032142570027
Saved artifacts/fraud_model.pkl
